In [275]:
import pandas as pd
import numpy as np
import datetime as dt

from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


In [276]:
df=pd.read_csv('/content/ITSM_data.csv')
df#.iloc[46603]

<ipython-input-276-2c22a2b0204e>:1: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.



,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4.0,0.601292,...,NaN,4/11/2013 13:50,4/11/2013 13:51,"3,87,16,91,111",Other,1.0,SD0000007,2.0,NaN,NaN
1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3.0,0.415050,...,2/12/2013 12:31,2/12/2013 12:36,2/12/2013 12:36,"4,35,47,86,389",Software,1.0,SD0000011,1.0,NaN,NaN
2,DTA000024,application,Desktop Application,WBS000092,IM0000006,Closed,NS,3,NaN,0.517551,...,NaN,13-01-2014 15:12,13-01-2014 15:13,"4,84,31,19,444",No error - works as designed,1.0,SD0000017,NaN,NaN,NaN
3,WBA000124,application,Web Based Application,WBS000088,IM0000011,Closed,4,4,4.0,0.642927,...,NaN,14-11-2013 09:31,14-11-2013 09:31,"4,32,18,33,333",Operator error,1.0,SD0000025,NaN,NaN,NaN
4,WBA000124,application,Web Based Application,WBS000088,IM0000012,Closed,4,4,4.0,0.345258,...,NaN,8/11/2013 13:55,8/11/2013 13:55,"3,38,39,03,333",Other,1.0,SD0000029,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46601,SBA000464,application,Server Based Application,WBS000073,IM0047053,Closed,4,4,4.0,0.231896,...,NaN,31-03-2014 16:29,31-03-2014 16:29,"0,095",Other,1.0,SD0147021,NaN,NaN,NaN
46602,SBA000461,application,Server Based Application,WBS000073,IM0047054,Closed,4,4,4.0,0.805153,...,NaN,31-03-2014 15:29,31-03-2014 15:29,"0,428333333",User error,1.0,SD0146967,NaN,NaN,NaN
46603,LAP000019,computer,Laptop,WBS000091,IM0047055,Closed,5,5,5.0,0.917466,...,NaN,31-03-2014 15:32,31-03-2014 15:32,"0,071666667",Hardware,1.0,SD0146982,NaN,NaN,NaN
46604,WBA000058,application,Web Based Application,WBS000073,IM0047056,Closed,4,4,4.0,0.701278,...,NaN,31-03-2014 15:42,31-03-2014 15:42,"0,116944444",Software,1.0,SD0146986,NaN,NaN,NaN


In [277]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46606 entries, 0 to 46605
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CI_Name                     46606 non-null  object 
 1   CI_Cat                      46495 non-null  object 
 2   CI_Subcat                   46495 non-null  object 
 3   WBS                         46606 non-null  object 
 4   Incident_ID                 46606 non-null  object 
 5   Status                      46606 non-null  object 
 6   Impact                      46606 non-null  object 
 7   Urgency                     46606 non-null  object 
 8   Priority                    45226 non-null  float64
 9   number_cnt                  46606 non-null  float64
 10  Category                    46606 non-null  object 
 11  KB_number                   46606 non-null  object 
 12  Alert_Status                46606 non-null  object 
 13  No_of_Reassignments         466

In [278]:
df.CI_Subcat.replace('Desktop','Desktop Application',inplace=True)
df.CI_Subcat.replace(['Application Server','Virtual Tape Server','ESX Server','zOS Server','Neoview Server','X86 Server',
                       'Unix Server','Oracle Server','Windows Server in extern beheer','Thin Client','NonStop Server',
                       'Number','Windows Server','Linux Server',np.nan,'SharePoint Farm','Lines'],
                       'Server Based Application',inplace=True)
df.CI_Subcat.replace('RAC Service','Banking Device',inplace=True)
df.CI_Subcat.replace(['Iptelephony','Protocol','Net Device','IPtelephony','ESX Cluster','Standard Application'],
                       'Web Based Application',inplace=True)
df.CI_Subcat.replace(['VMWare','Security Software','zOS Systeem','Firewall','Database Software','VDI','Instance',
                       'MQ Queue Manager','Automation Software','Citrix','SAP','Encryption'],'System Software',inplace=True)
df.CI_Subcat.replace(['UPS','Omgeving'],'Client Based Application',inplace=True)
df.CI_Subcat.replace(['NonStop Storage','NonStop Harddisk','Tape Library','zOS Cluster','DataCenterEquipment',
                       'MigratieDummy'],'Database',inplace=True)
df.CI_Subcat.replace(['Modem','Router'],'Network Component',inplace=True)
df.CI_Subcat.replace('KVM Switches','Switch',inplace=True)

In [279]:
df.CI_Cat.replace(['database'],'storage',inplace=True)
df.CI_Cat.replace(['applicationcomponent'],'subapplication',inplace=True)
df.CI_Cat.replace(['displaydevice','officeelectronics','Phone','networkcomponents'],'hardware',inplace=True)
df.CI_Cat.replace(np.nan,'application',inplace=True)

In [280]:
df.drop(['Related_Change','No_of_Related_Changes','No_of_Related_Incidents'], axis=1, inplace= True)

df.dropna(subset=['Priority'], inplace=True)

df.dropna(subset=['Resolved_Time'], inplace=True)
df.dropna(subset=['No_of_Reassignments'], inplace=True)

#df['Handle_Time_hrs'] = df['Handle_Time_hrs'].apply(lambda x: x.total_seconds() / 3600) 

In [281]:
df['Reopen_Time'].isna().sum()

41406

In [282]:
df['Impact'].replace(np.nan,-1,inplace=True)
df['Impact'].value_counts()
df['Impact']=df['Impact'].astype(int)

In [283]:
df['Urgency'].value_counts()
df['Urgency'].replace('5 - Very Low',5,inplace=True)
df['Urgency']=df['Urgency'].astype(int)

In [284]:
df['CI_Name'].value_counts()

SUB000456    3007
SBA000263    2155
SBA000607    1721
SBA000462    1681
WBA000058    1581
             ... 
LAP000812       1
NET000036       1
WBA000028       1
LAP000887       1
SWT000008       1
Name: CI_Name, Length: 2640, dtype: int64

In [285]:
df['Resolved_Time']=pd.to_datetime(df['Resolved_Time'])
df['Resolved_Time_year']=df['Resolved_Time'].dt.year
df['Resolved_Time_month']=df['Resolved_Time'].dt.month

df['Resolved_Time_year']#=df['Resolved_Time_year'].astype(int)
df['Resolved_Time_month']=df['Resolved_Time_month'].astype(int)

In [286]:
df['Open_Time']=pd.to_datetime(df['Open_Time'])
df['Open_Time_year']=df['Open_Time'].dt.year
df['Open_Time_month']=df['Open_Time'].dt.month

df['Open_Time_year']=df['Open_Time_year'].astype(int)
df['Open_Time_month']=df['Open_Time_month'].astype(int)

In [287]:
'''
df['Close_Time']=pd.to_datetime(df['Close_Time'])
df['Close_Time_year']=df['Close_Time'].dt.year
df['Close_Time_month']=df['Close_Time'].dt.month

df['Close_Time_year']=df['Close_Time_year'].astype(int)
df['Close_Time_month']=df['Close_Time_month'].astype(int)
'''

"\ndf['Close_Time']=pd.to_datetime(df['Close_Time'])\ndf['Close_Time_year']=df['Close_Time'].dt.year\ndf['Close_Time_month']=df['Close_Time'].dt.month\n\ndf['Close_Time_year']=df['Close_Time_year'].astype(int)\ndf['Close_Time_month']=df['Close_Time_month'].astype(int)\n"

In [288]:
#df['Reopen_Time']=pd.to_datetime(df['Reopen_Time'])
#df['Reopen_Time_year']=df['Reopen_Time'].dt.year
#df['Reopen_Time_month']=df['Reopen_Time'].dt.month

#df['Reopen_Time_year']=df['Reopen_Time_year'].astype(int)
#df['Reopen_Time_month']=df['Reopen_Time_month'].astype(int)

In [289]:
#df['Open_Time'] = pd.to_datetime(df['Open_Time'])
#df['Resolved_Time'] = pd.to_datetime(df['Resolved_Time'])

# Calculate the time difference in hours between 'Open_Time' and 'Resolved_Time'
df['Time_to_Resolve'] = (df['Resolved_Time'] - df['Open_Time']).dt.total_seconds() / 3600
df

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,Resolved_Time_year,Resolved_Time_month,Open_Time_year,Open_Time_month,Time_to_Resolve
0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4.0,0.601292,...,4/11/2013 13:51,"3,87,16,91,111",Other,1.0,SD0000007,2013,4,2012,5,8256.300000
1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3.0,0.415050,...,2/12/2013 12:36,"4,35,47,86,389",Software,1.0,SD0000011,2013,2,2012,12,1700.866667
3,WBA000124,application,Web Based Application,WBS000088,IM0000011,Closed,4,4,4.0,0.642927,...,14-11-2013 09:31,"4,32,18,33,333",Operator error,1.0,SD0000025,2013,11,2012,7,11637.700000
4,WBA000124,application,Web Based Application,WBS000088,IM0000012,Closed,4,4,4.0,0.345258,...,8/11/2013 13:55,"3,38,39,03,333",Other,1.0,SD0000029,2013,8,2012,10,7370.900000
5,WBA000124,application,Web Based Application,WBS000088,IM0000013,Closed,4,4,4.0,0.006676,...,8/11/2013 13:54,"3,38,34,36,944",Other,1.0,SD0000031,2013,8,2012,10,7370.450000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46601,SBA000464,application,Server Based Application,WBS000073,IM0047053,Closed,4,4,4.0,0.231896,...,31-03-2014 16:29,"0,095",Other,1.0,SD0147021,2014,3,2014,3,0.100000
46602,SBA000461,application,Server Based Application,WBS000073,IM0047054,Closed,4,4,4.0,0.805153,...,31-03-2014 15:29,"0,428333333",User error,1.0,SD0146967,2014,3,2014,3,0.433333
46603,LAP000019,computer,Laptop,WBS000091,IM0047055,Closed,5,5,5.0,0.917466,...,31-03-2014 15:32,"0,071666667",Hardware,1.0,SD0146982,2014,3,2014,3,0.066667
46604,WBA000058,application,Web Based Application,WBS000073,IM0047056,Closed,4,4,4.0,0.701278,...,31-03-2014 15:42,"0,116944444",Software,1.0,SD0146986,2014,3,2014,3,0.116667


In [290]:
df['Priority'].value_counts()

4.0    22154
5.0    16200
3.0     4811
2.0      413
1.0        3
Name: Priority, dtype: int64

In [291]:
grouped_subcats = df.groupby('CI_Subcat')['WBS'].apply(set)
single_wbs_subcats = grouped_subcats[grouped_subcats.apply(lambda wbs: len(wbs) == 1)]

single_wbs_subcats.apply(lambda wbs: print(f"CI_Subcat: {single_wbs_subcats.name}, WBS: {list(wbs)[0]}"))

CI_Subcat: WBS, WBS: WBS000091
CI_Subcat: WBS, WBS: WBS000091
CI_Subcat: WBS, WBS: WBS000096


CI_Subcat
Keyboard    None
Monitor     None
Printer     None
Name: WBS, dtype: object

In [292]:
X = df.loc[:,['CI_Cat','CI_Subcat','WBS','Category']]
y = df.Priority

X = pd.get_dummies(X)
X.head(2)

,CI_Cat_application,CI_Cat_computer,CI_Cat_hardware,CI_Cat_software,CI_Cat_storage,CI_Cat_subapplication,CI_Subcat_Banking Device,CI_Subcat_Client Based Application,CI_Subcat_Controller,CI_Subcat_Database,...,WBS_WBS000329,WBS_WBS000330,WBS_WBS000331,WBS_WBS000332,WBS_WBS000335,WBS_WBS000337,WBS_WBS000339,Category_complaint,Category_incident,Category_request for information
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


#Predict High Priority

In [293]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43581 entries, 0 to 46605
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   CI_Name                     43581 non-null  object        
 1   CI_Cat                      43581 non-null  object        
 2   CI_Subcat                   43581 non-null  object        
 3   WBS                         43581 non-null  object        
 4   Incident_ID                 43581 non-null  object        
 5   Status                      43581 non-null  object        
 6   Impact                      43581 non-null  int64         
 7   Urgency                     43581 non-null  int64         
 8   Priority                    43581 non-null  float64       
 9   number_cnt                  43581 non-null  float64       
 10  Category                    43581 non-null  object        
 11  KB_number                   43581 non-null  object    

#Grab the best score to predict Priority

In [294]:
df["Open_Time"] = pd.to_datetime(df["Open_Time"])
df["Date"] = df["Open_Time"].dt.date
print(df)

         CI_Name          CI_Cat                 CI_Subcat        WBS  \
0      SUB000508  subapplication     Web Based Application  WBS000162   
1      WBA000124     application     Web Based Application  WBS000088   
3      WBA000124     application     Web Based Application  WBS000088   
4      WBA000124     application     Web Based Application  WBS000088   
5      WBA000124     application     Web Based Application  WBS000088   
...          ...             ...                       ...        ...   
46601  SBA000464     application  Server Based Application  WBS000073   
46602  SBA000461     application  Server Based Application  WBS000073   
46603  LAP000019        computer                    Laptop  WBS000091   
46604  WBA000058     application     Web Based Application  WBS000073   
46605  DCE000077        hardware                  Database  WBS000267   

      Incident_ID  Status  Impact  Urgency  Priority  number_cnt  ...  \
0       IM0000004  Closed       4        4       4

In [295]:
res = (pd.to_datetime(df['Date'])
       .dt.floor('d')
       .value_counts()
       .rename_axis('dates')
       .reset_index(name='count'))
res.rename(columns = {'dates':'ds', 'count':'y'}, inplace = True)
res

,ds,y
0,2013-11-18,537
1,2014-01-27,472
2,2014-03-02,445
3,2014-01-21,442
4,2013-10-14,439
...,...,...
318,2013-08-16,1
319,2013-03-05,1
320,2013-08-15,1
321,2013-10-05,1


In [296]:
m = Prophet()
m.fit(res)
future = m.make_future_dataframe(periods=365)
future.tail()

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpz6wm0tiv/amm3bb4g.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpz6wm0tiv/ajwj23kz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=22399', 'data', 'file=/tmp/tmpz6wm0tiv/amm3bb4g.json', 'init=/tmp/tmpz6wm0tiv/ajwj23kz.json', 'output', 'file=/tmp/tmpz6wm0tiv/prophet_modelw3wqzoic/prophet_model-20230519141944.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:19:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:19:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


,ds
683,2015-11-29
684,2015-11-30
685,2015-12-01
686,2015-12-02
687,2015-12-03


In [297]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
683,2015-11-29,438.331225,267.566986,607.673623
684,2015-11-30,539.505658,375.563230,711.169249
685,2015-12-01,552.424947,388.187675,722.271109
686,2015-12-02,517.951638,353.422419,681.937412
687,2015-12-03,531.670175,369.300729,699.703062


In [298]:
plot_plotly(m, forecast)
plot_components_plotly(m, forecast)

In [299]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)
'''
logreg=LogisticRegression()
logreg.fit(X_train,y_train)

dico = {'penalty':['l2', 'elasticnet', 'None'], 'class_weight': ['dict','balanced']} #min_samples_leaf': range(1,16), 'min_samples_split': [2, 5, 7, 10, 15, 30]}
# initialize a grid model
grid = GridSearchCV(logreg, dico)
# train the model
grid.fit(X,y)

print("best score:",grid.best_score_)
print("best parameters:",grid.best_params_)
print("best estimator:",grid.best_estimator_)
'''

'\nlogreg=LogisticRegression()\nlogreg.fit(X_train,y_train)\n\ndico = {\'penalty\':[\'l2\', \'elasticnet\', \'None\'], \'class_weight\': [\'dict\',\'balanced\']} #min_samples_leaf\': range(1,16), \'min_samples_split\': [2, 5, 7, 10, 15, 30]}\n# initialize a grid model\ngrid = GridSearchCV(logreg, dico)\n# train the model\ngrid.fit(X,y)\n\nprint("best score:",grid.best_score_)\nprint("best parameters:",grid.best_params_)\nprint("best estimator:",grid.best_estimator_)\n'

In [300]:
from sklearn.tree import DecisionTreeClassifier

dct = DecisionTreeClassifier()

dct.fit(X_train,y_train)

dico = {'criterion': ['entropy', 'log_loss']}#, 'min_samples_split': [2, 5, 7, 10, 15, 30]}
# initialize a grid model
grid = GridSearchCV(dct, dico)
# train the model
grid.fit(X,y)

print("best score:",grid.best_score_)
print("best parameters:",grid.best_params_)
print("best estimator:",grid.best_estimator_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning:

The least populated class in y has only 3 members, which is less than n_splits=5.



best score: 0.8322435764164098
best parameters: {'criterion': 'entropy'}
best estimator: DecisionTreeClassifier(criterion='entropy')


In [301]:
dct = DecisionTreeClassifier(criterion='entropy')

dct=dct.fit(X_train,y_train)

ypredict=dct.predict(X_test)

ypredict

array([4., 4., 3., ..., 5., 4., 4.])